In [ ]:
import warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress TensorFlow info and warning messages
warnings.filterwarnings('ignore', category=DeprecationWarning)


Let’s start with something simple: the stack of two layers we used in the previous section. Its Functional API version looks like the following listing.

In [2]:
import keras
from keras import layers, models

#create an input layer (symbolic tensor)
inputs = keras.Input(shape=(3,), name="my_input")
#create a layer and call it with the input
features = layers.Dense(64, activation="relu")(inputs)

outputs = layers.Dense(10, activation="softmax")(features)

model = keras.Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ my_input (InputLayer)           │ (None, 3)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 906 (3.54 KB)

 Trainable params: 906 (3.54 KB)

 Non-trainable params: 0 (0.00 B)

the functional API is a way to create models that are more flexible than the Sequential API.

they hold more complex topologies, such as multi-input models and multi-output models.

example: 

Let’s say you’re building a system to rank customer support tickets by priority and route them to the appropriate department. Your model has three inputs:

- The title of the ticket (text)
- The text of the ticket (text)
- The priority of the ticket (categorical)

the model has two outputs:
- The priority of the ticket (categorical)
- The department to route the ticket to (categorical)

In [4]:
vocabulary_size = 10000
num_tags = 100
num_departments = 4

#define model inputs
title = keras.Input(shape=(vocabulary_size,), name="title")
text_body = keras.Input(shape=(vocabulary_size,), name="text_body")
tags = keras.Input(shape=(num_tags,), name="tags")

#combine input features into a single tensor
features = layers.Concatenate()([title, text_body, tags])
#apply a intermidiate layer to recombine input features into richer representation
features = layers.Dense(64, activation="relu")(features)

#define model outputs
priority = layers.Dense(1, activation="sigmoid", name="priority")(features)
department = layers.Dense(
	num_departments, activation="softmax", name="department")(features)

model = keras.Model(inputs=[title, text_body, tags], outputs=[priority, department])

### TRAINING A MULTI-INPUT, MULTI-OUTPUT MODEL

In [6]:
import numpy as np

num_samples = 1280

title_data = np.random.randint(0, 2, size=(num_samples, vocabulary_size))
text_body_data = np.random.randint(0, 2, size=(num_samples, vocabulary_size))
tags_data = np.random.randint(0, 2, size=(num_samples, num_tags))

priority_data = np.random.random(size=(num_samples, 1))
department_data = np.random.randint(0, 2, size=(num_samples, num_departments))

model.compile(
	optimizer="rmsprop",
	loss=["mean_squared_error", "categorical_crossentropy"],
	metrics=[["mean_absolute_error"], ["accuracy"]]
)

model.fit(
	[title_data, text_body_data, tags_data],
	[priority_data, department_data],
	epochs=3
)

model.evaluate([title_data, text_body_data, tags_data], [priority_data, department_data])

priority_preds, department_preds = model.predict([title_data, text_body_data, tags_data])

Epoch 1/3
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - department_accuracy: 0.2633 - department_loss: 63.4675 - loss: 63.7987 - priority_loss: 0.3312 - priority_mean_absolute_error: 0.4994
Epoch 2/3
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - department_accuracy: 0.2602 - department_loss: 64.6681 - loss: 64.9993 - priority_loss: 0.3312 - priority_mean_absolute_error: 0.4994
Epoch 3/3
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - department_accuracy: 0.2703 - department_loss: 71.5123 - loss: 71.8435 - priority_loss: 0.3312 - priority_mean_absolute_error: 0.4994
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - department_accuracy: 0.1352 - department_loss: 97.2811 - loss: 97.6123 - priority_loss: 0.3312 - priority_mean_absolute_error: 0.4994 
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


You can plot a Functional model as a graph with the plot_model()

In [7]:
import keras
import pydot
import graphviz
import tensorflow as tf

keras.utils.plot_model( model, "ticket_classifier_with_shape_info.png", show_shapes=True)

You must install graphviz (see instructions at https://graphviz.gitlab.io/download/) for `plot_model` to work.


In [8]:
model.layers

[<InputLayer name=title, built=True>,
 <InputLayer name=text_body, built=True>,
 <InputLayer name=tags, built=True>,
 <Concatenate name=concatenate_1, built=True>,
 <Dense name=dense_3, built=True>,
 <Dense name=priority, built=True>,
 <Dense name=department, built=True>]

In [9]:
model.layers[3].input

[<KerasTensor shape=(None, 10000), dtype=float32, sparse=False, ragged=False, name=title>,
 <KerasTensor shape=(None, 10000), dtype=float32, sparse=False, ragged=False, name=text_body>,
 <KerasTensor shape=(None, 100), dtype=float32, sparse=False, ragged=False, name=tags>]

In [10]:
model.layers[3].output

<KerasTensor shape=(None, 20100), dtype=float32, sparse=False, ragged=False, name=keras_tensor_6>

feature extraction, creating models that reuse intermediate features from another model.

Let’s say you want to add another output to the previous model—you want to estimate how long a given issue ticket will take to resolve, a kind of difficulty rating. You could do this via a classification layer over three categories: “quick,” “medium,” and “difficult.” You don’t need to recreate and retrain a model from scratch. You can start from the intermediate features of your previous model, since you have access to them, like this.

In [16]:
features = model.layers[4].output #layers[4] is the output of the previous model
difficulty = layers.Dense(3, activation="softmax", name="difficulty")(features)

new_model=keras.Model(
	inputs=[title, text_body, tags],
	outputs=[priority, department, difficulty]
)


You can train your model in much the same way as you would train a Sequential model, by calling fit() with lists of input and output data. These lists of data should be in the same order as the inputs you passed to the Model constructor.

In [17]:
keras.utils.plot_model( new_model, "updated_ticket_classifier.png", show_shapes=True)

You must install graphviz (see instructions at https://graphviz.gitlab.io/download/) for `plot_model` to work.


### Subclassing the Model class

The last model-building pattern you should know about is the most advanced one: Model subclassing. You learned in chapter 3 how to subclass the Layer class to create custom layers. Subclassing Model is pretty similar:

Subclassing Model is pretty similar: 
- In the __init__() method, define the layers the model will use.
- In the call() method, define the forward pass of the model, reusing the layers previously created. 
- Instantiate your subclass, and call it on data to create its weights.

REWRITING OUR PREVIOUS EXAMPLE AS A SUBCLASSED MODEL

In [18]:
class CustomerTicketModel(keras.Model):
    def __init__(self, num_departments):
        super().__init__()
        self.concat_layer = layers.Concatenate()
        self.mixing_layer = layers.Dense(64, activation="relu")
        self.priority_scorer = layers.Dense(1, activation="sigmoid")
        self.department_classifier = layers.Dense(
            num_departments, activation="softmax"
        )

    #define forward pass in the call method
    def call(self, inputs):
        title = inputs["title"]
        text_body = inputs["text_body"]
        tags = inputs["tags"]

        features = self.concat_layer([title, text_body, tags])
        features = self.mixing_layer(features)
        priority = self.priority_scorer(features) #output 1
        department = self.department_classifier(features) #output 2
        return priority, department
    

Once you’ve defined the model, you can instantiate it. Note that it will only create its weights the first time you call it on some data, much like Layer subclasses:

In [19]:
model = CustomerTicketModel(num_departments=4)

In [20]:
priority, department = model(
    {"title": title_data, "text_body": text_body_data, "tags": tags_data}
)

What, then, is the difference between a Layer subclass and a Model subclass? It’s simple: a “layer” is a building block you use to create models, and a “model” is the top-level object that you will actually train, export for inference, etc. In short, a Model has fit(), evaluate(), and predict() methods. Layers don’t. Other than that, the two classes are virtually identical. (Another difference is that you can save a model to a file on disk)

You can compile and train a Model subclass just like a Sequential or Functional model:

In [21]:
model.compile(optimizer="rmsprop",
    loss=["mean_squared_error", "categorical_crossentropy"],
    metrics=[["mean_absolute_error"], ["accuracy"]])

model.fit({"title": title_data,
        "text_body": text_body_data,
        "tags":tags_data},
        [priority_data, department_data],
        epochs=1)

model.evaluate({"title": title_data,
                "text_body": text_body_data,
                "tags":tags_data},
                [priority_data, department_data])

priority_preds, department_preds = model.predict(
{"title": title_data, "text_body": text_body_data, "tags": tags_data})



40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.2937 - categorical_crossentropy_loss: 31.1573 - loss: 31.4721 - mean_absolute_error: 0.4808 - mean_squared_error_loss: 0.3148
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5664 - categorical_crossentropy_loss: 35.3319 - loss: 35.6525 - mean_absolute_error: 0.4873 - mean_squared_error_loss: 0.3206
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


The Model subclassing workflow is the most flexible way to build a model. It enables you to build models that cannot be expressed as directed acyclic graphs of layersimagine, for instance, a model where the call() method uses layers inside a for loop, or even calls them recursively. Anything is possible—you’re in charge.

Functional and subclassed models are also substantially different in nature. A Functional model is an explicit data structure—a graph of layers, which you can view, inspect, and modify. A subclassed model is a piece of bytecode—a Python class with a call() method that contains raw code. This is the source of the subclassing workflow’s flexibility—you can code up whatever functionality you like—but it introduces new limitations. For instance, because the way layers are connected to each other is hidden inside the body of the call() method, you cannot access that information. Calling summary() will not display layer connectivity, and you cannot plot the model topology via plot_model(). Likewise, if you have a subclassed model, you cannot access the nodes of the graph of layers to do feature extraction because there is simply no graph. Once the model is instantiated, its forward pass becomes a complete black box.

### 7.2.4 Mixing and matching different components

you can use a subclassed layer or model in a Functional model.

In [24]:
class Classifier(keras.Model):
    def __init__(self, num_classes=2):
        super().__init__()
        #se é classficiacao binaria ou multiclasse
        if num_classes ==2: 
            num_units = 1
            activation = "sigmoid"
        else:
            num_units = num_classes
            activation = "softmax"

        self.dense = layers.Dense(num_units, activation=activation)
    def call(self, inputs):
        return self.dense(inputs)

inputs = keras.Input(shape=(3,))
features = layers.Dense(64, activation="relu") (inputs)
outputs = Classifier(num_classes=10) (features)
model = keras.Model(inputs=inputs, outputs=outputs)

Inversely, you can use a Functional model as part of a subclassed layer or model

In [25]:
inputs = keras.Input(shape=(64,))
outputs = layers.Dense(1, activation="sigmoid")(inputs) 
binary_classifier = keras.Model(inputs=inputs, outputs=outputs)

class MyModel(keras.Model):
    def __init__(self, num_classes=2):
        super().__init__()
        self.dense = layers.Dense(64, activation="relu")
        self.classifier = binary_classifier  
    
    def call(self, inputs): 
        features = self.dense(inputs) 
        return self.classifier(features)  
model = MyModel()

### 7.3 Using built-in training and evaluation loops

In [4]:
import keras
from keras import layers
from keras.datasets import mnist

def get_mnist_model():
    inputs = keras.Input(shape=(28*28,))
    features = layers.Dense(512, activation="relu")(inputs)
    features = layers.Dropout(0.5) (features)
    outputs = layers.Dense(10, activation="softmax") (features)

    model = keras.Model(inputs, outputs)
    return model

(images, labels), (test_images, test_labels) = mnist.load_data()
images = images.reshape((60000, 28*28)).astype("float32") / 255 
test_images = test_images.reshape((10000, 28*28)).astype("float32") / 255 
train_images, val_images = images[10000:], images[:10000]
train_labels, val_labels = labels[10000:], labels[:10000]

model = get_mnist_model()
model.compile(optimizer="rmsprop",
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"])
model.fit(train_images, train_labels,
        epochs=3,
        validation_data=(val_images,val_labels))
#compute loss and metrics on new data
test_metrics = model.evaluate(test_images, test_labels)
#compute classification probabilities on new data
predictions = model.predict(test_images)

Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9117 - loss: 0.2964 - val_accuracy: 0.9565 - val_loss: 0.1499
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9528 - loss: 0.1647 - val_accuracy: 0.9678 - val_loss: 0.1123
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9637 - loss: 0.1292 - val_accuracy: 0.9704 - val_loss: 0.1081
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9734 - loss: 0.1041
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 855us/step


There are a couple of ways you can customize this simple workflow: 
- Provide your own custom metrics.
- Pass callbacks to the fit() method to schedule actions to be taken at specific points during training.

### 7.3.1 Writing your own metrics

In [10]:
import tensorflow as tf

class RootMeanSquaredError(keras.metrics.Metric):
    #Define the state  variables in the  constructor.
    #Like for layers, you  have access to  the add_weight() method.
    def __init__(self, name="rmse", **kwargs):
        super().__init__(name=name, **kwargs)
        self.mse_sum = self.add_weight(name="mse_sum", initializer="zeros")
        self.total_samples = self.add_weight(
            name="total_samples", initializer="zeros", dtype="int32"
        )

    #implement the state update logic in update_state(). the y_true argument
    #is the targets for one batch, while y_pred represents the
    # corresponding predicitons from the model. 
    # sample_weights wont be used here 
    def update_state(self, y_true, y_pred, sample_weight=None):
    #to match our MNIST modal,we expect categorical predicitons and integer labels
        y_true = tf.one_hot(y_true, depth=tf.shape(y_pred)[1])
        mse = tf.reduce_sum(tf.square(y_true - y_pred))
        self.mse_sum.assign_add(mse)
        num_samples = tf.shape(y_pred) [0]
        self.total_samples.assign_add(num_samples)
    #You use the result() method to return the current value of the metric:
    def result(self):
        return tf.sqrt(self.mse_sum / tf.cast(self.total_samples, tf.float32))

    #Meanwhile, you also need to expose a way
    #  to reset the metric state without
    #  having to reinstantiate it—this
    #  enables the same metric objects to be
    #  used across different epochs of training 
    # or across both training and evaluation.
    def reset_state(self):
        self.mse_sum.assign(0.)
        self.total_samples.assign(0)

Custom metrics can be used just like built-in ones. Let’s test-drive our own metric:

In [11]:
model = get_mnist_model()
model.compile(optimizer="rmsprop",
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy", RootMeanSquaredError()])
    
model.fit(train_images, train_labels,
        epochs=3,
        validation_data=(val_images, val_labels))
test_metrics = model.evaluate(test_images, test_labels)

Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9115 - loss: 0.2984 - rmse: 0.3652 - val_accuracy: 0.9590 - val_loss: 0.1416 - val_rmse: 0.2493
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9540 - loss: 0.1625 - rmse: 0.2666 - val_accuracy: 0.9667 - val_loss: 0.1159 - val_rmse: 0.2254
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9632 - loss: 0.1302 - rmse: 0.2393 - val_accuracy: 0.9689 - val_loss: 0.1095 - val_rmse: 0.2152
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9707 - loss: 0.1067 - rmse: 0.2133


### 7.3.2 Using callbacks